In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "sh2orc/Llama-3.1-Korean-8B-Instruct"

SYSTEM_PROMPT = """
당신은 지휘관의 결심을 돕는 군사 의사결정 보조 AI입니다.

역할:
- 주어진 전장 상황, 아군 전력 상태, 임무를 토대로 가능한 여러 개의 행동방안(COA)을 제안합니다.
- 각 COA에 대해: 의도, 개념(주요 행동), 필요한 전력, 장점, 단점, 주요 위험요인, 전제조건을 정리합니다.
- 구체적인 표적/좌표/사격지시 등 "직접적인 교전 지시"는 절대 하지 않습니다.
- 항상 "최종 결심은 인간 지휘관이 한다"는 점을 명확히 합니다.

출력 형식:
1. 상황 요약
2. COA 1
3. COA 2
4. COA 3
5. COA 비교 및 고려사항

가능한 한 간결하고 실무적인 한국어로 답변하세요.
""".strip()


def load_model():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    return tokenizer, model


def generate_response(tokenizer, model, user_input: str, max_new_tokens: int = 512):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove prompt prefix if it appears in the decoded text
    if decoded.startswith(prompt):
        decoded = decoded[len(prompt):]

    return decoded.strip()


if __name__ == "__main__":
    tokenizer, model = load_model()

    user_input = """
적 전차 2대와 전투기 1대가 관측되었고, 전체 추정 전력은 약 2개 여단 규모로 평가된다.
우리 부대는 기계화 보병대대 1개(전투력 80%), 보병중대 1개(전투력 60%), 대전차 화력은 중간 수준,
방공전력은 제한적이다. 상급부대 임무는 '12시간 동안 적의 돌파를 지연하고 전투력을 보존할 것'이다.
지형은 개활지와 소규모 구릉이 혼재하며, 우리 후방에는 중요한 교량 1개가 있다.

이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.
""".strip()

    print("=== 지휘관 입력 ===")
    print(user_input)
    print("\n=== AI 응답 ===\n")
    print(generate_response(tokenizer, model, user_input))


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

MODEL_NAME = "sh2orc/Llama-3.1-Korean-8B-Instruct"

SYSTEM_PROMPT = """
당신은 지휘관의 결심을 돕는 군사 의사결정 보조 AI입니다.

역할:
- 주어진 전장 상황, 아군 전력 상태, 임무를 토대로 가능한 여러 개의 행동방안(COA)을 제안합니다.
- 각 COA에 대해: 의도, 개념(주요 행동), 필요한 전력, 장점, 단점, 주요 위험요인, 전제조건을 정리합니다.
- 구체적인 표적/좌표/사격지시 등 "직접적인 교전 지시"는 절대 하지 않습니다.
- 항상 "최종 결심은 인간 지휘관이 한다"는 점을 명확히 합니다.

출력 형식:
1. 상황 요약
2. COA 1
3. COA 2
4. COA 3
5. COA 비교 및 고려사항

가능한 한 간결하고 실무적인 한국어로 답변하세요.
""".strip()


def load_model():
    """모델과 토크나이저를 로드합니다. (캐시가 되어 있다면 빠르게 로드됩니다)"""
    print(f"모델 로딩 중: {MODEL_NAME}")
    start_time = time.time()
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    end_time = time.time()
    print(f"모델 로딩 완료. 소요 시간: {end_time - start_time:.2f}초\n")
    return tokenizer, model


def generate_response(tokenizer, model, user_input: str, max_new_tokens: int = 512):
    """단일 사용자 입력에 대한 응답을 생성합니다."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # 응답에서 프롬프트 접두사를 제거합니다.
    if decoded.startswith(prompt):
        decoded = decoded[len(prompt):]

    return decoded.strip()


if __name__ == "__main__":
    # 1. 모델 로드
    tokenizer, model = load_model()

    # 2. 비교할 시나리오 목록 정의
    # 각 시나리오를 문자열로 리스트에 저장합니다.
    scenarios = [
        # 시나리오 1: 원본 (방어/지연 임무)
        """
        적 전차 2대와 전투기 1대가 관측되었고, 전체 추정 전력은 약 2개 여단 규모로 평가된다.
        우리 부대는 기계화 보병대대 1개(전투력 80%), 보병중대 1개(전투력 60%), 대전차 화력은 중간 수준,
        방공전력은 제한적이다. 상급부대 임무는 '12시간 동안 적의 돌파를 지연하고 전투력을 보존할 것'이다.
        지형은 개활지와 소규모 구릉이 혼재하며, 우리 후방에는 중요한 교량 1개가 있다.
        이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.
        """.strip(),
        
        # 시나리오 2: 전력 우세 및 공격 임무 (임무와 전력 변경)
        """
        적 전력은 보병중대 1개(전투력 70%)와 경장갑차 3대가 관측되었다.
        우리 부대는 기계화 보병대대 2개(전투력 95%), 강력한 포병 및 항공 지원이 가능하다.
        상급부대 임무는 '2시간 이내에 목표 지역을 확보하고 적의 잔존 세력을 섬멸할 것'이다.
        지형은 시가지와 산악 지형이 혼재하며, 목표 지역은 시가지 내의 주요 통신 시설이다.
        이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.
        """.strip(),
        
        # 시나리오 3: 비정규전 및 인도적 지원 임무 (전력/임무/상황 모두 변경)
        """
        적은 소규모 비정규군(게릴라) 형태로 활동하며, 민간인 지역에 숨어있다.
        우리 부대는 경보병대대 1개(전투력 90%), 의료팀 및 공병 지원 전력이 있다.
        상급부대 임무는 '민간인 피해를 최소화하며 구호 물자를 전달하고, 비정규군의 주 보급로를 차단할 것'이다.
        지형은 밀집된 주거 지역과 외곽의 농지이다. 아군에게는 민간인 피해 최소화가 최우선이다.
        이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.
        """.strip(),
    ]

    # 3. 각 시나리오에 대해 응답 생성 및 비교 출력
    for i, scenario in enumerate(scenarios, 1):
        print(f"============== 시나리오 {i} : {time.strftime('%H:%M:%S')} ==============")
        
        # 간결한 시나리오 요약 출력
        print(f"--- 지휘관 입력 (요약) ---")
        # 입력의 처음 두 줄만 출력하여 상황을 빠르게 파악하도록 함
        lines = scenario.split('\n')
        print('\n'.join(lines[:2]).strip()) 
        print(f"...") 
        
        start_time = time.time()
        response = generate_response(tokenizer, model, scenario)
        end_time = time.time()
        
        print("\n--- AI 응답 ---\n")
        print(response)
        print(f"\n--- 응답 생성 완료. 소요 시간: {end_time - start_time:.2f}초 ---")
        print("=================================================================\n")

모델 로딩 중: sh2orc/Llama-3.1-Korean-8B-Instruct


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


모델 로딩 완료. 소요 시간: 6.46초

============== 시나리오 1 : 12:59:53 ==============
--- 지휘관 입력 (요약) ---
적 전차 2대와 전투기 1대가 관측되었고, 전체 추정 전력은 약 2개 여단 규모로 평가된다.
        우리 부대는 기계화 보병대대 1개(전투력 80%), 보병중대 1개(전투력 60%), 대전차 화력은 중간 수준,
...

--- AI 응답 ---

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

당신은 지휘관의 결심을 돕는 군사 의사결정 보조 AI입니다.

역할:
- 주어진 전장 상황, 아군 전력 상태, 임무를 토대로 가능한 여러 개의 행동방안(COA)을 제안합니다.
- 각 COA에 대해: 의도, 개념(주요 행동), 필요한 전력, 장점, 단점, 주요 위험요인, 전제조건을 정리합니다.
- 구체적인 표적/좌표/사격지시 등 "직접적인 교전 지시"는 절대 하지 않습니다.
- 항상 "최종 결심은 인간 지휘관이 한다"는 점을 명확히 합니다.

출력 형식:
1. 상황 요약
2. COA 1
3. COA 2
4. COA 3
5. COA 비교 및 고려사항

가능한 한 간결하고 실무적인 한국어로 답변하세요.user

적 전차 2대와 전투기 1대가 관측되었고, 전체 추정 전력은 약 2개 여단 규모로 평가된다.
        우리 부대는 기계화 보병대대 1개(전투력 80%), 보병중대 1개(전투력 60%), 대전차 화력은 중간 수준,
        방공전력은 제한적이다. 상급부대 임무는 '12시간 동안 적의 돌파를 지연하고 전투력을 보존할 것'이다.
        지형은 개활지와 소규모 구릉이 혼재하며, 우리 후방에는 중요한 교량 1개가 있다.
        이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.assistant

**상황 요약**

* 적 전차 2대와 전투기 1대가 관측됨(전체 추정 전력 약 2개 여단)
* 우리 부